In [ ]:
import pandas as pd
import numpy as np
from plotly.graph_objs import *
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, EffectScatter

In [ ]:
df = pd.read_csv("ETH_daily.csv")  

In [ ]:
df["Close_1_shift"] = df["Close"].shift(1)  

In [ ]:
# 计算True Range

df["TR"] = np.abs(df.High - df.Low)           
df["TR"] = np.maximum(
    df["TR"],
    np.maximum(
        np.abs(df.Close_1_shift - df.High),
        np.abs(df.Close_1_shift - df.Low),
    ),
)

In [ ]:
# 计算Average True Range-ATR

n_array = np.array(df["TR"].values)
n_array[20] = np.mean(df["TR"][:20])
for i in range(21, df.shape[0]):
    n_array[i] = (19.0 * n_array[i - 1] + df["TR"][i]) / 20.0
df["N"] = n_array


In [ ]:
# Compute upper and lower bounds based on Turtle Algorithm

df["upper_bound"] = df["High"].shift(1).rolling(window=20).max()
df["lower_bound"] = df["Low"].shift(1).rolling(window=10).min()


In [ ]:
money = [1000.0 for i in range(20)]  # initial money value in USDT
crypto_value = [0 for i in range(20)]
crypto = 0  
fees = 0.001  # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  


In [ ]:
# 简单版海龟交易，不考虑仓位管理

for i in range(21, df.shape[0]):  

    if ( crypto == 0 and df["Close_1_shift"].iloc[i] > df["upper_bound"].iloc[i-1] ): # 1.突破上轨，建仓

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        crypto = np.round(money[-1]*(1.0 - fees)/ price, 4)
        crypto_value += [crypto*df["Close"].iloc[i]]
        money += [0]
        positions += [{ "date": df.Date.iloc[i], "price": price}]
        stop_loss = price - 2.0 * df["N"].iloc[i]  # set stop loss


    elif crypto == 0:                                       # 2.空仓不动
        money += [money[-1]]
        crypto_value += [0]
        
    elif crypto > 0 and (
        df["Close_1_shift"].iloc[i]< df["lower_bound"].iloc[i-1]  # 3、突破下轨或达到止损条件stop_loss或最后一次交易，清仓
        or df["Close_1_shift"].iloc[i] < stop_loss  
        or i == df.shape[0] - 1 ): 

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [crypto * price *(1-fees)]
        crypto = 0
        crypto_value +=[0]
        stop_loss = 0.0

        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        positions = []
        
    elif crypto > 0:                              # 4.持仓不动
        money += [0]
        crypto_value += [crypto * df["Close"].iloc[i]]

In [ ]:
success_rate = 0
if len(success_history) + len(failure_history) > 0:
    success_rate = len(success_history) / (len(failure_history) + len(success_history))


In [ ]:
print("-" * 50)
print("Benchmark",(df.Close.iloc[df.shape[0]-1]/df.Close.iloc[0]-1)*1000+1000)
print("Capital at the end", np.round(money[-1], 2))
print("Accumulated return at the end", np.round(money[-1]/money[0]*100, 4),"%")
print("Win ratio", success_rate )
print("-" * 50)
print("Summary of % change in positions")
percent_change = []
positions = []
for h in [failure_history, success_history]:
    for position in h:
        percent_change.append(np.round((position["price"][1] - position["price"][0])/ position["price"][0]* 100.0, 2,))
        print("Percent change in position", percent_change[-1])
        positions += [position]
        
print("-" * 50)
print("Total trade times", len(percent_change))
print("Average return per trade", np.round(sum(percent_change)/len(percent_change),2),"%")
print("Max win", max(percent_change),"%")
print("Max loss", min(percent_change),"%")

In [ ]:
attr=[str(t) for t in df.Date]
bd = [str(i["date"][0]) for i in positions]
sd = [str(i["date"][1]) for i in positions]
bp = [np.round((i["price"][0]),2) for i in positions]
sp = [np.round((i["price"][1]),2) for i in positions]

v1=np.array(df.loc[:,['Open','Close','Low','High']]).tolist()
v2=np.array(df.upper_bound).tolist()
v3=np.array(df.lower_bound).tolist()
kline1 = (
    Kline()
    .add_xaxis(attr)
    .add_yaxis("ETH",v1)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="simple_turtle"),
    ) 
)

line1 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("upper_bound",v2 , is_smooth = True,)
    .add_yaxis("lower_bound",v3 , is_smooth = True,)
)
cb = (
    EffectScatter()
    .add_xaxis(bd)
    .add_yaxis("Buy", bp)
)

cs = (
    EffectScatter()
    .add_xaxis(sd)
    .add_yaxis("Sell", sp)
)

kline1.overlap(line1)
kline1.overlap(cb)
kline1.overlap(cs)
kline1.render("p1_simple_turtle.html")


In [ ]:
rtn = [np.round((crypto_value[i]+money[i])/money[0],4) for i in range(df.shape[0]-1)]
ben = [np.round(j/df.Close.iloc[0],4) for j in df.Close]

line2 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("rtn",rtn , is_smooth = True,)
    .add_yaxis("ben",ben , is_smooth = True,) 
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="simple_turtle"),
    ) 
)
line2.render("p2_simple_turtle.html")